In [3]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from valhalla import Actor, get_config


def get_isochrone(query: dict):
    result = {}
    isochrones = actor.isochrone(query)
    for contour_ix, isochrone in enumerate(isochrones["features"]):
        geom = isochrone["geometry"]["coordinates"]
        time = isochrone["properties"]["contour"]
        result[time] = Polygon(geom)
    return result


def get_walk_isochrones(location: dict[str, float], times: list[int] = [5, 10, 15, 20]):
    query = {
        "locations": [location],
        "costing": "pedestrian",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_bike_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "bicycle",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)


def get_auto_isochrones(location: dict[str, float], times: list[int] = [10, 15, 30]):
    query = {
        "locations": [location],
        "costing": "auto",
        "contours": [{"time": i} for i in times],
    }

    return get_isochrone(query)

In [4]:
# CITY = "budapest"
# # CSV filename in ../data/stops/{CITY}/
# STOPS = "stops_gtfs_15min.csv"

CITY = "madrid"
# CSV filename in ../data/stops/{CITY}/
STOPS = "madrid_stops.csv"

In [5]:
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)

In [6]:
config = get_config(
    tile_extract=f"../data/valhalla/{CITY}/valhalla_tiles.tar",
    verbose=True,
)

config["service_limits"]["isochrone"]["max_contours"] = 10
config["service_limits"]["isochrone"]["max_locations"] = 10_000
config["service_limits"]["isochrone"]["max_distance"] = 100_000

# instantiate Actor to load graph and call actions
actor = Actor(config)

2025/01/30 15:12:22.498333 [INFO] Tile extract successfully loaded with tile count: 47
2025/01/30 15:12:22.498407 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/01/30 15:12:22.498411 [WARN] Traffic tile extract could not be loaded


In [7]:
stops = pd.read_csv(f"../data/stops/{CITY}/{STOPS}", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.to_crs(23700, inplace=True)
stops.head(3)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,parent_station,wheelchair_boarding,stop_direction,clust,max_distance,quant,geometry
0,7357,Tour Eiffel,48.859075,2.293369,NaN,NaN,7357,2.0,-64.0,11,0.0,1,POINT (-570807.872 521754.602)


In [8]:
locations = []
for row in stops.itertuples():
    locations.append({"lon": row.stop_lon, "lat": row.stop_lat})

In [9]:
isochones_wide = []
isochones = []
for row in stops.itertuples():
    w = get_walk_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    b = get_bike_isochrones(
        {"lon": row.stop_lon, "lat": row.stop_lat}, times=[5, 10, 15]
    )
    # a = get_auto_isochrones({"lon": row.stop_lon, "lat": row.stop_lat}, times=[10, 15])
    # isochones_wide.append(
    #     [row.stop_id, w[5], w[10], w[15], b[5], b[10], b[15], a[10], a[15]]
    # )
    isochones.append([row.stop_id, w[5], "walk", 5])
    isochones.append([row.stop_id, w[10], "walk", 10])
    isochones.append([row.stop_id, w[15], "walk", 15])
    isochones.append([row.stop_id, b[5], "bicycle", 5])
    isochones.append([row.stop_id, b[10], "bicycle", 10])
    isochones.append([row.stop_id, b[15], "bicycle", 15])
    # isochones.append([row.stop_id, a[10], "car", 10])
    # isochones.append([row.stop_id, a[15], "car", 15])
isochones_wide = pd.DataFrame.from_records(
    isochones_wide,
    columns=[
        "stop_id",
        "walk_5",
        "walk_10",
        "walk_15",
        "bike_5",
        "bike_10",
        "bike_15",
        # "car_10",
        # "car_15",
    ],
)
isochones = pd.DataFrame.from_records(
    isochones, columns=["stop_id", "geometry", "costing", "range"]
)

In [10]:
isochones.head(10)

,stop_id,geometry,costing,range
0,7357,"POLYGON ((2.297369 48.862165, 2.293916 48.8616...",walk,5
1,7357,"POLYGON ((2.294369 48.86559, 2.291369 48.86537...",walk,10
2,7357,"POLYGON ((2.296369 48.869104, 2.295369 48.8691...",walk,15
3,7357,"POLYGON ((2.296369 48.869822, 2.295369 48.8700...",bicycle,5
4,7357,"POLYGON ((2.300369 48.881566, 2.298037 48.8810...",bicycle,10
5,7357,"POLYGON ((2.302369 48.892281, 2.301295 48.8921...",bicycle,15


In [11]:
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file(f"../output/{CITY}/isochrones.geojson")

In [12]:
# isochones = pd.read_csv(f"../output/{CITY}/isochrones.csv")
# print(isochones.head(6).to_markdown(index=False))